### Assingnment 3 - IPL cricket database

#### Abstract
      
     The main aim of this assignment is to convert the SQL database which was created in the assignment 2 to NoSQL. The     converted database contains all the domain related data from the assignment 2. The NoSQL database uses Mongodb. 
 

In [ ]:
# Importing libraries
import pymongo 
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import pandas as pd
import numpy as np
import tweepy as tw
from pandas import DataFrame

In [50]:
# Reading the CSV file from the previous assignment 
match = pd.read_csv('Match.csv', index_col=0)
match.head()

,match_id,Team1,Team2,match_date,Season_Year,Venue_Name,City_Name,Country_Name,Toss_Winner,match_winner,Toss_Name,Win_Type,Outcome_Type,ManOfMach,Win_Margin
Match_SK,,,,,,,,,,,,,,,
0,335987,Royal Challengers Bangalore,Kolkata Knight Riders,4/18/2008,2008,M Chinnaswamy Stadium,Bangalore,India,Royal Challengers Bangalore,Kolkata Knight Riders,field,runs,Result,BB McCullum,140.0
1,335988,Kings XI Punjab,Chennai Super Kings,4/19/2008,2008,"Punjab Cricket Association Stadium, Mohali",Chandigarh,India,Chennai Super Kings,Chennai Super Kings,bat,runs,Result,MEK Hussey,33.0
2,335989,Delhi Daredevils,Rajasthan Royals,4/19/2008,2008,Feroz Shah Kotla,Delhi,India,Rajasthan Royals,Delhi Daredevils,bat,wickets,Result,MF Maharoof,9.0
3,335990,Mumbai Indians,Royal Challengers Bangalore,4/20/2008,2008,Wankhede Stadium,Mumbai,India,Mumbai Indians,Royal Challengers Bangalore,bat,wickets,Result,MV Boucher,5.0
4,335991,Kolkata Knight Riders,Deccan Chargers,4/20/2008,2008,Eden Gardens,Kolkata,India,Deccan Chargers,Kolkata Knight Riders,bat,wickets,Result,DJ Hussey,5.0


In [33]:
# Connecting to the mongodb database
mongodb_host= 'localhost'
mongodb_port = '27017'

client = MongoClient(mongodb_host + ':' + mongodb_port)
db = client['cricket']

print (db)


Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'cricket')


In [80]:
# Using Twitter API to fetch details 
consumer_key = 'Ulg0aOEDEy09m90eQ1WjgY8ax'
consumer_secret = 'hJWMDqHG6rAtNn5fV6oIgNwaVcdy2yTIYOWXNWoyOKDD8l1lUZ'
access_token = '715063474020159488-TWPJ4GzyfYJ94Iy4ItvIu4c7Qd065nD'
access_token_secret = '2mgrQXQ5m6yG1zmZcUBluykXg5LGrgw4KZYeNKPtDCicI'


auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

search_words = "%#cricket"
date_since = "2018-11-16"


tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(100)
tweets



In [39]:
# Reading the tweets
tweets = [tweet.text for tweet in tweets]
tweets

['RT @Saj_PakPassion: Shoaib Akhtar\xa0has suggested that whenever possible a three-match ODI series between Pakistan &amp; India\xa0should be played t…',
 'Wisden has just depressed me by highlighting that Steve Smith scored eleven Test centuries against us in the 2010s.… https://t.co/7X5hAeYCNg',
 'RT @SunilAh64145529: Howzat #Cricket 😂 https://t.co/3vnLCuQHNZ',
 'RT @timesofindia: #Cricket  #TheAshes 🏏\n\n2005 Ashes was one of the all time great cricket series: @RickyPonting \n\nRead: https://t.co/ACofoC…',
 'RT @Saj_PakPassion: Shoaib Akhtar\xa0has suggested that whenever possible a three-match ODI series between Pakistan &amp; India\xa0should be played t…',
 'RT @TheCricketWire: @windiescricket @Russell12A 🏏\n\nBest Strike Rates in IPL History \n(30+ Batting Average) \n\n1⃣ Andre Russell 186.41\n2⃣ Ris…',
 "RT @Daniel86criket: Waqas, Apologies for spoiling your fame night buddy but the future of Pakistan isn't looking bright. #Cricket https://t…",
 'RT @MarkDuffett: A Pair of Aces

In [40]:
# Splitting the tweets
user_list = [tweet.split(':')[0] for tweet in tweets ]
tweets_list = [tweet.split(':')[-1] for tweet in tweets ]

tweets_list

[' Shoaib Akhtar\xa0has suggested that whenever possible a three-match ODI series between Pakistan &amp; India\xa0should be played t…',
 '//t.co/7X5hAeYCNg',
 '//t.co/3vnLCuQHNZ',
 '//t.co/ACofoC…',
 ' Shoaib Akhtar\xa0has suggested that whenever possible a three-match ODI series between Pakistan &amp; India\xa0should be played t…',
 ' @windiescricket @Russell12A 🏏\n\nBest Strike Rates in IPL History \n(30+ Batting Average) \n\n1⃣ Andre Russell 186.41\n2⃣ Ris…',
 '//t…',
 '//t.co/rNm9yqRIZh #Cricket',
 '//t.co/WbXskQ1jt2',
 ' Shoaib Akhtar\xa0has suggested that whenever possible a three-match ODI series between Pakistan &amp; India\xa0should be played t…',
 ' Which cricket format is the best? #Cricket',
 '//t.co/yAbK7smh0r',
 ' Michael Clarke\xa0"Australia &amp; probably every other team sucked up to India &amp; were too scared to sledge Kohli &amp; other Indi…',
 ' Shoaib Akhtar\xa0has suggested that whenever possible a three-match ODI series between Pakistan &amp; India\xa0should be 

In [41]:
# Reading the tweets
tweets_list

[' Shoaib Akhtar\xa0has suggested that whenever possible a three-match ODI series between Pakistan &amp; India\xa0should be played t…',
 '//t.co/7X5hAeYCNg',
 '//t.co/3vnLCuQHNZ',
 '//t.co/ACofoC…',
 ' Shoaib Akhtar\xa0has suggested that whenever possible a three-match ODI series between Pakistan &amp; India\xa0should be played t…',
 ' @windiescricket @Russell12A 🏏\n\nBest Strike Rates in IPL History \n(30+ Batting Average) \n\n1⃣ Andre Russell 186.41\n2⃣ Ris…',
 '//t…',
 '//t.co/rNm9yqRIZh #Cricket',
 '//t.co/WbXskQ1jt2',
 ' Shoaib Akhtar\xa0has suggested that whenever possible a three-match ODI series between Pakistan &amp; India\xa0should be played t…',
 ' Which cricket format is the best? #Cricket',
 '//t.co/yAbK7smh0r',
 ' Michael Clarke\xa0"Australia &amp; probably every other team sucked up to India &amp; were too scared to sledge Kohli &amp; other Indi…',
 ' Shoaib Akhtar\xa0has suggested that whenever possible a three-match ODI series between Pakistan &amp; India\xa0should be 

In [44]:
# Putting the tweets in DataFrame
df = DataFrame ({'user_ID':user_list,'tweets':tweets_list})
df.head()

,user_ID,tweets
0,RT @Saj_PakPassion,Shoaib Akhtar has suggested that whenever pos...
1,Wisden has just depressed me by highlighting t...,//t.co/7X5hAeYCNg
2,RT @SunilAh64145529,//t.co/3vnLCuQHNZ
3,RT @timesofindia,//t.co/ACofoC…
4,RT @Saj_PakPassion,Shoaib Akhtar has suggested that whenever pos...


### Answering the questions 

#### 1) What are tags are associated with a person, place or thing?

In [54]:
# Tags associated with match
Tags_Match = match.loc[:,['Team1','Team2','match_date']]
Tags_Match

,Team1,Team2,match_date
Match_SK,,,
0,Royal Challengers Bangalore,Kolkata Knight Riders,4/18/2008
1,Kings XI Punjab,Chennai Super Kings,4/19/2008
2,Delhi Daredevils,Rajasthan Royals,4/19/2008
3,Mumbai Indians,Royal Challengers Bangalore,4/20/2008
4,Kolkata Knight Riders,Deccan Chargers,4/20/2008
...,...,...,...
632,Delhi Daredevils,Royal Challengers Bangalore,5/14/2017
633,Mumbai Indians,Rising Pune Supergiants,5/16/2017
634,Sunrisers Hyderabad,Kolkata Knight Riders,5/17/2017


In [55]:
# Tags associated with venue
Tags_venue = match.loc[:,['Venue_Name','City_Name','Country_Name']]
Tags_venue

,Venue_Name,City_Name,Country_Name
Match_SK,,,
0,M Chinnaswamy Stadium,Bangalore,India
1,"Punjab Cricket Association Stadium, Mohali",Chandigarh,India
2,Feroz Shah Kotla,Delhi,India
3,Wankhede Stadium,Mumbai,India
4,Eden Gardens,Kolkata,India
...,...,...,...
632,Feroz Shah Kotla,Delhi,India
633,Wankhede Stadium,Mumbai,India
634,M Chinnaswamy Stadium,Bengaluru,India


In [56]:
# Tags associated with Result
Tags_Result = match.loc[:,['match_winner','Win_Type','Outcome_Type','Win_Margin']]
Tags_venue

,Venue_Name,City_Name,Country_Name
Match_SK,,,
0,M Chinnaswamy Stadium,Bangalore,India
1,"Punjab Cricket Association Stadium, Mohali",Chandigarh,India
2,Feroz Shah Kotla,Delhi,India
3,Wankhede Stadium,Mumbai,India
4,Eden Gardens,Kolkata,India
...,...,...,...
632,Feroz Shah Kotla,Delhi,India
633,Wankhede Stadium,Mumbai,India
634,M Chinnaswamy Stadium,Bengaluru,India


 ### 3.What people, places or things are trending in your domain?

In [86]:
X = df.groupby('user_ID').agg({'tweets':np.size}).sort_values(['tweets'],ascending=False)
X.head(5)

,tweets
user_ID,
RT @Saj_PakPassion,19
RT @Daniel86criket,8
RT @KKRiders,7
RT @briancjeeves,7
RT @PoppingCreaseSA,3


It can be seen that the user_ID named @Saj_PakPassion has tweeted 19 times in our gathered. Hence being the most most trending person in our domain.

### 4.What people, places or things are popular in your domain?

In [82]:
# 
y =match.groupby('City_Name').agg({'match_id':np.size}).sort_values(['match_id'],ascending=False)
y.head(5)

,match_id
City_Name,
Mumbai,85
Kolkata,61
Delhi,60
Bangalore,58
Chennai,48


It can be seen that the city_name with Mumbai has hosted most number of matches. Hence,it is the most popular place in the domain 

### Design Explanations
  In SQL databases we will storing values in the form of relational tables by linking each table with the primary key and foriegn key 
  relationship. But in the NoSQL databases will be storing the data in the form of documents. 
 
  In the SQL databases, the values are stored in the form of tables but in the NoSQL databases the values stored in the form of 
  Collections. For Example, in our cricket database we will be having data for each match in a table and all its related data in the other table. But in the NoSQL databases all data are stand alone document.       
    

### Report 
 
    The data used in this Assignment is downloaded from Kaggle. The data is audited for null values. Also, Additionally an 
    Twitter API was used to fetch tweets and the user IDs with the hashtag similar to cricket. 
    
    Web API 
    We extracted data using the below Twitter API 
    consumer_key = 'Ulg0aOEDEy09m90eQ1WjgY8ax'
    consumer_secret = 'hJWMDqHG6rAtNn5fV6oIgNwaVcdy2yTIYOWXNWoyOKDD8l1lUZ'
    access_token = '715063474020159488-TWPJ4GzyfYJ94Iy4ItvIu4c7Qd065nD'
    access_token_secret = '2mgrQXQ5m6yG1zmZcUBluykXg5LGrgw4KZYeNKPtDCicI'
    
    Importing libraries : : requests to get data from site, pandas for data manipulation and analysis and creating Data frames, json to convert the python dictionary into a JSON string that can be written into a file, The OS module provides or allows you to interface with the underlying operating system that Python is running on - windows or Mac.
    
    Raw csv data We extracted data from csv file using read_csv in pandas. To get in detail knowledge about the database, we used below methods: info(), isnull(), shape, columns, describe, iloc etc. iloc, in pandas is used to select rows and columns by number, in the order that they appear in the data frame.All the questions asked in the assignment were answered with the help of this Raw CSV file.
    
### conclusion

    I understood the concept the NoSQl databases and the general procedure for converting the SQL database to NoSQL databases.
    Also, got used the familiar concept of Dataframe, Sorting the value in the forom descending value. Grouping the tweets by 
    most number of tweets by a single twitter Used_ID and the use of aggregate function.
    
### Contributions

    Self efforts : 30%

    External source: 30%

    Guidance by the Professor : 20%

    Guidance by Teaching Assistant: 20%

### Citations 

     Data source : https://www.kaggle.com/raghu07/ipl-data-till-2017
     https://www.w3schools.com/python/python_mongodb_drop_collection.asp
     https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
     https://www.youtube.com/watch?v=Y0zpUOYStys
     https://github.com/nikbearbrown/INFO_6210/blob/master/Project/Jobs_DB_Project/IPYNB/Project%202%20NoSQL%20.ipynb
     
### LICENSE:

Copyright 2020 Kumaran Nehru Uthra. Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions: The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software. THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

     
     